In [1]:
import os
import pandas as pd

In [2]:
base_dir = "../data/"

In [3]:
df = pd.read_json(os.path.join(base_dir, "full_compared_df.json"))

In [4]:
df.shape[0]

1182

In [5]:
df.columns

Index(['size', 'connections', 'read_percent', 'write_percent',
       'write_query_id', 'read_query_id', 'template', 'queries_no_p',
       'tps_no_p', 'process_file_no_p', 'queries_p', 'tps_p', 'process_file_p',
       'count_in_group'],
      dtype='object')

In [6]:
df_double = df.loc[df["count_in_group"] > 1 & df["count_in_group"].notna()]

In [7]:
df_double.shape[0]

1166

In [15]:
df_double.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1166 entries, 0 to 1181
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   size               1166 non-null   int64 
 1   connections        1166 non-null   int64 
 2   read_percent       1166 non-null   int64 
 3   write_percent      1166 non-null   int64 
 4   write_query_id     1166 non-null   int64 
 5   read_query_id      1166 non-null   int64 
 6   template           1166 non-null   object
 7   queries_no_p       1166 non-null   object
 8   tps_no_p           1166 non-null   object
 9   process_file_no_p  1166 non-null   object
 10  queries_p          1166 non-null   object
 11  tps_p              1166 non-null   object
 12  process_file_p     1166 non-null   object
 13  count_in_group     1166 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 136.6+ KB


## формирую фичи эффективности

In [17]:
df_double["effective"] = df_double["tps_p"].astype(float) > df_double["tps_no_p"].astype(float)

/tmp/ipykernel_38306/3883617989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["effective"] = df_double["tps_p"].astype(float) > df_double["tps_no_p"].astype(float)


In [18]:
df_double["effective"].value_counts(dropna=False)

effective
True     790
False    376
Name: count, dtype: int64

In [19]:
df_double["effective_coef"] = df_double["tps_p"].astype(float) / df_double["tps_no_p"].astype(float)

/tmp/ipykernel_38306/3530313600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["effective_coef"] = df_double["tps_p"].astype(float) / df_double["tps_no_p"].astype(float)


In [27]:
df_double.loc[(df_double["size"] == 100)\
   & (df_double["read_percent"] < 100 )\
   # & (df_double["template"] == "query_222222.sql.tpl" )\
   & (df_double["effective_coef"] < 1.0)].head()

,size,connections,read_percent,write_percent,write_query_id,read_query_id,template,queries_no_p,tps_no_p,process_file_no_p,queries_p,tps_p,process_file_p,count_in_group,effective,effective_coef
52,100,100,36,66,1,0,query_111111.sql.tpl,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",5151.94476,./data/output/test_res_id_32.txt,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",4553.333854,./data/output/test_res_id_33.txt,2,False,0.883809
53,100,100,73,27,1,0,query_111111.sql.tpl,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",5540.099949,./data/output/test_res_id_34.txt,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",5456.251435,./data/output/test_res_id_35.txt,2,False,0.984865
54,100,100,92,8,1,0,query_111111.sql.tpl,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",5061.393373,./data/output/test_res_id_36.txt,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",3394.174551,./data/output/test_res_id_37.txt,2,False,0.670601
64,100,320,36,66,1,0,query_111111.sql.tpl,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",3087.084146,./data/output/test_res_id_164.txt,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",2975.374359,./data/output/test_res_id_165.txt,2,False,0.963814
188,100,90,36,66,1,0,query_111111.sql.tpl,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",153.006126,./data/output/test_res_id_56.txt,"[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_...",102.89688,./data/output/test_res_id_57.txt,2,False,0.672502


In [52]:
df_double["effective_coef"].describe()

count    1166.000000
mean        1.303747
std         0.684521
min         0.284239
25%         0.934648
50%         1.175874
75%         1.447375
max         8.038749
Name: effective_coef, dtype: float64

## добавляю фичи памяти

**было включено кэша на 2 гб на инстансах бд и 3 гб на gppool в каждом тесте**

In [53]:
df_double["size"].value_counts(dropna=False)

size
300    228
100    218
120    212
280    212
150    192
200     95
400      9
Name: count, dtype: int64

In [55]:
maping_size2memory_consumption = {
    "100": 1.3,
    "120": 1.55,
    "150": 1.95,
    "200": 2.57,
    "280": 3.6,
    "300": 3.85,
    "400": 5.15
}

def maper_func_memory(size, base_size):
    real_mem = maping_size2memory_consumption[str(int(size))]
    return real_mem / base_size

In [56]:
df_double["db_mem_coef"] = df_double["size"].apply(maper_func_memory, base_size=2.0)

/tmp/ipykernel_57815/423166093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["db_mem_coef"] = df_double["size"].apply(maper_func_memory, base_size=2.0)


In [57]:
df_double["pgpool_cache_coef"] = df_double["size"].apply(maper_func_memory, base_size=3.0)

/tmp/ipykernel_57815/4236952757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["pgpool_cache_coef"] = df_double["size"].apply(maper_func_memory, base_size=3.0)


In [60]:
df_double[["db_mem_coef","pgpool_cache_coef"]].describe()

,db_mem_coef,pgpool_cache_coef
count,1166.000000,1166.000000
mean,1.251244,0.834162
std,0.527981,0.351987
min,0.650000,0.433333
25%,0.775000,0.516667
50%,0.975000,0.650000
75%,1.800000,1.200000
max,2.575000,1.716667


## Формирую признак сложности

In [63]:
df_double["queries_no_p"].iloc[0].

[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_accounts AS a JOIN pgbench_branches AS b ON a.bid = b.bid JOIN pgbench_tellers AS t ON b.bid = t.bid WHERE a.aid = :aid;',
  'analyze': 'SELECT a.*, b.*, t.* FROM pgbench_accounts AS a JOIN pgbench_branches AS b ON a.bid = b.bid JOIN pgbench_tellers AS t ON b.bid = t.bid WHERE a.aid = 10;',
  'explain': 'cost=8.61..29.05 rows=10 width=813',
  'explain_file': './data/output/explain_id_24_query_id_0.txt'}]

In [75]:
def get_explain_info(row):
    queries_dict = row["queries_no_p"]
    read_id = row["read_query_id"]
    write_id = row ["write_query_id"]
    read_exp = 0.0
    write_exp = 0.0
    try:
        read_query_dict = queries_dict[read_id]
        match = re.search(r"\.\.(\d+\.\d+)", read_query_dict["explain"])
        if match:
            read_exp = float(match.group(1))
    except IndexError:
        pass
    try:
        write_query_dict = queries_dict[write_id]
        match = re.search(r"\.\.(\d+\.\d+)", write_query_dict["explain"])
        if match:
            write_exp = float(match.group(1))
    except IndexError:
        pass
    return read_exp, write_exp

In [76]:
df_double["read_cost"], df_double["write_cost"] = zip(
    *df_double[["queries_no_p","read_query_id","write_query_id"]].apply(get_explain_info, axis=1)
)

/tmp/ipykernel_57815/3448010084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["read_cost"], df_double["write_cost"] = zip(


In [77]:
df_double[["read_cost", "write_cost"]].describe()

,read_cost,write_cost
count,1166.000000,1166.000000
mean,427.778328,1395.123842
std,1765.437988,7524.922895
min,0.010000,0.000000
25%,18.410000,8.460000
50%,32.550000,13.090000
75%,61.857500,289.272500
max,34414.940000,100081.720000


## Записываю фичи в csv

In [79]:
df_double.columns

Index(['size', 'connections', 'read_percent', 'write_percent',
       'write_query_id', 'read_query_id', 'template', 'queries_no_p',
       'tps_no_p', 'process_file_no_p', 'queries_p', 'tps_p', 'process_file_p',
       'count_in_group', 'effective', 'effective_coef', 'db_mem_coef',
       'pgpool_cache_coef', 'read_cost', 'write_cost'],
      dtype='object')

In [80]:
df_need = df_double[['connections', 'read_percent', 'write_percent',
                     'db_mem_coef', 'pgpool_cache_coef', 'read_cost', 'write_cost',
                     'effective', 'effective_coef']]

In [81]:
df_need.columns

Index(['connections', 'read_percent', 'write_percent', 'db_mem_coef',
       'pgpool_cache_coef', 'read_cost', 'write_cost', 'effective',
       'effective_coef'],
      dtype='object')

In [83]:
df_need.head()

,connections,read_percent,write_percent,db_mem_coef,pgpool_cache_coef,read_cost,write_cost,effective,effective_coef
0,80,100,0,0.65,0.433333,29.05,0.0,True,1.085585
1,90,100,0,0.65,0.433333,29.05,0.0,True,1.030422
2,100,100,0,0.65,0.433333,29.05,0.0,True,1.032450
3,150,100,0,0.65,0.433333,29.05,0.0,True,2.429909
4,210,100,0,0.65,0.433333,29.05,0.0,True,2.218325


In [85]:
df_need.to_csv(os.path.join(base_dir, "prepared_df.csv"), index=False)